# Step1-1 维基百科中文语料的处理


使用WikiExtractor提取维基百科语料

WikiExtractor链接：https://github.com/attardi/wikiextractor
需要的环境官网有写明：The tool is written in Python and requires Python 2.7 or Python 3.3+ but no additional library.
意思是:基于python2.7或python3.3， 且不需要依赖于第三方模块。
安装GitHub，之后打开命令行：

git clone https://github.com/attardi/wikiextractor wikiextractor

cd wikiextractor

python setup.py install

将语料的压缩文件拷贝进wikiextractor文件夹里面，接下来就是核心命令：
打开命令行：

cd wikiextractor

python WikiExtractor.py -b 1024M -o extracted zhwiki-20200201-pages-articles-multistream.xml.bz2

# Step1-2: 汉语新闻语料库的处理

In [13]:
!pip3 install gensim

  Using cached gensim-3.8.1-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl (24.7 MB)
  Using cached smart_open-1.9.0.tar.gz (70 kB)
  Using cached boto-2.49.0-py2.py3-none-any.whl (1.4 MB)
  Using cached boto3-1.12.5-py2.py3-none-any.whl (128 kB)
     |████████████████████████████████| 5.9 MB 30 kB/s eta 0:00:0152     |██████████████████████████      | 4.8 MB 12 kB/s eta 0:01:32
     |████████████████████████████████| 69 kB 14 kB/s eta 0:00:011
     |████████████████████████████████| 547 kB 35 kB/s eta 0:00:01
    Running setup.py install for smart-open ... done


In [1]:
import pandas as pd
import jieba
import re
from collections import Counter

def token(string):
    return re.findall('\w+', string)

In [18]:
#读取汉语新闻语料库文件
df = pd.read_csv('sqlResult_1558435.csv', encoding='GB2312')
df.head()

UnicodeDecodeError: 'gb2312' codec can't decode byte 0xac in position 6: illegal multibyte sequence

In [17]:
import chardet

binary_data = open('sqlResult_1558435.csv', 'rb').read()
chardet.detect(binary_data)

{'encoding': 'GB2312', 'confidence': 0.99, 'language': 'Chinese'}

In [ ]:
TOKEN = []
df.fillna('', inplace=True)
articles = df['comment'].tolist()
print('number of articels: %d'%len(articles))
for i in range(len(articles)):
    if i % 10000 == 0: print(i)
    TOKEN += jieba.cut(''.join(token(articles[i])))

In [ ]:
TOKEN

In [ ]:
for i, line in enumerate((open('sqlResult_1558435.csv', encoding='GB2312'))):
    if i % 1000 == 0: print(i)
    TOKEN += jieba.cut(''.join(token(line)))

In [ ]:
words_count = Counter(TOKEN)
words_count.most_common(20)

In [ ]:
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]
words_count2 = Counter(TOKEN_2_GRAM)
words_count2.most_common(20)

In [ ]:
def prob_2(word1, word2):
    if word1 in words_count:
        if word1 + word2 in words_count2: 
            return words_count2[word1+word2] / words_count[word1]
        else:
            return 1 / words_count[word1]
    else:
        return 1/ len(words_count)
    
def get_probability(sentence):
    words = list(jieba.cut(sentence))
    
    sentence_prob = 1
    
    for i, word in enumerate(words[:-1]):
        word = words[i]
        next_ = words[i+1]
        
        probability = prob_2(word, next_)
        
        sentence_prob *= probability
    
    return sentence_prob